In [1]:
import pandas as pd
import re
import pickle
import json
from sklearn.preprocessing import LabelEncoder
import numpy as np
np.random.seed(42)
import random
random.seed(42)
from transformers import AutoTokenizer

In [2]:
dire = '/home/ubuntu/dataset_em/'

In [3]:
ls /home/ubuntu/dataset_em/amazon_google/

tableA.csv  tableB.csv  test.csv  train.csv  valid.csv


In [4]:
ta = pd.read_csv(dire + 'amazon_google/tableA.csv').fillna('')
ta['cluster_id_left'] = ta['manufacturer'].tolist()
tb = pd.read_csv(dire + 'amazon_google/tableB.csv').fillna('')
tb['cluster_id_right'] = tb['manufacturer'].tolist()

ttr = pd.read_csv(dire + 'amazon_google/train.csv')
ttr.rename(columns={"ltable_id": "id_left", "rtable_id": "id_right"},inplace = True)
tval = pd.read_csv(dire + 'amazon_google/valid.csv')
tval.rename(columns={"ltable_id": "id_left", "rtable_id": "id_right"},inplace = True)
tte = pd.read_csv(dire + 'amazon_google/test.csv')
tte.rename(columns={"ltable_id": "id_left", "rtable_id": "id_right"},inplace = True)
ta.rename(columns={"id": "id_left", "title": "title_left","manufacturer":"manufacturer_left", "price":"price_left"},
         inplace = True)
tb.rename(columns={"id": "id_right", "title": "title_right","manufacturer":"manufacturer_right", "price":"price_right"},
         inplace = True)

In [6]:
ta.head()

,id_left,title_left,manufacturer_left,price_left,cluster_id_left
0,0,clickart 950 000 premier image pack ( dvd-rom ),broderbund,,broderbund
1,1,ca international arcserve lap/desktop oem 30pk,computer associates,,computer associates
2,2,noah 's ark activity center ( jewel case ages ...,victory multimedia,,victory multimedia
3,3,peachtree by sage premium accounting for nonpr...,sage software,599.99,sage software
4,4,singing coach unlimited,carry-a-tune technologies,99.99,carry-a-tune technologies


In [7]:
enc = LabelEncoder()
cluster_id_set_left = set()
cluster_id_set_left.update(ta['cluster_id_left'].tolist())
cluster_id_set_right = set()
cluster_id_set_right.update(tb['cluster_id_right'].tolist())
cluster_id_set_left.update(cluster_id_set_right)
enc.fit(list(cluster_id_set_left))

LabelEncoder()

In [8]:
def merge_data(df_l, df_r, gs):
    res = gs.merge(df_l, on = 'id_left')
    res = res.merge(df_r, on = 'id_right') 
    return res

In [9]:
train = merge_data(ta, tb, ttr)
val = merge_data(ta, tb, tval)
test = merge_data(ta, tb, tte)

In [10]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def process_to_bert(dataset, attributes, tokenizer, comb_func, cutting_func=None, multi_encoder=None):
    dataset = dataset.fillna('')

    if multi_encoder is None:
        try:
            cluster_id_set_left = set()
            cluster_id_set_left.update(dataset['cluster_id_left'].tolist())
            cluster_id_set_right = set()
            cluster_id_set_right.update(dataset['cluster_id_right'].tolist())
            cluster_id_set_left.update(cluster_id_set_right)
            dataset = dataset.rename(columns={'cluster_id_left': 'label_multi1', 'cluster_id_right': 'label_multi2'})
            label_enc = LabelEncoder()
            label_enc.fit(list(cluster_id_set_left))
            dataset['label_multi1'] = label_enc.transform(dataset['label_multi1'])
            dataset['label_multi2'] = label_enc.transform(dataset['label_multi2'])

        except KeyError:
            pass
    else:
        dataset = dataset.rename(columns={'cluster_id_left': 'label_multi1', 'cluster_id_right': 'label_multi2'})
        try:
            dataset['label_multi1'] = multi_encoder.transform(dataset['label_multi1'])
            dataset['label_multi2'] = multi_encoder.transform(dataset['label_multi2'])
        except ValueError:
            dataset['label_multi1'] = 0
            dataset['label_multi2'] = 0

    print(f'Before cutting:')
    _print_attribute_stats(dataset, attributes)
    if cutting_func:
        tqdm.pandas(desc='Cutting attributes')
        dataset = dataset.progress_apply(cutting_func, axis=1)
        print(f'After cutting:')
        _print_attribute_stats(dataset, attributes)

    dataset['sequence_left'], dataset['sequence_left_titleonly'], dataset['sequence_right'], dataset[
        'sequence_right_titleonly'] = comb_func(dataset)

    dataset['sequence_left'] = dataset['sequence_left'].str.split()
    dataset['sequence_left'] = dataset['sequence_left'].str.join(' ')
    dataset['sequence_right'] = dataset['sequence_right'].str.split()
    dataset['sequence_right'] = dataset['sequence_right'].str.join(' ')

    dataset['sequence_left_titleonly'] = dataset['sequence_left_titleonly'].str.split()
    dataset['sequence_left_titleonly'] = dataset['sequence_left_titleonly'].str.join(' ')
    dataset['sequence_right_titleonly'] = dataset['sequence_right_titleonly'].str.split()
    dataset['sequence_right_titleonly'] = dataset['sequence_right_titleonly'].str.join(' ')

    tqdm.pandas(desc='Tokenizing left sequence for inspection')
    dataset['sequence_left_inspect'] = dataset['sequence_left'].progress_apply(lambda x: tokenizer.tokenize(x))
    dataset['sequence_left_titleonly_inspect'] = dataset['sequence_left_titleonly'].progress_apply(
        lambda x: tokenizer.tokenize(x))
    tqdm.pandas(desc='Tokenizing right sequence for inspection')
    dataset['sequence_right_inspect'] = dataset['sequence_right'].progress_apply(lambda x: tokenizer.tokenize(x))
    dataset['sequence_right_titleonly_inspect'] = dataset['sequence_right_titleonly'].progress_apply(
        lambda x: tokenizer.tokenize(x))

    dataset_combined_length = dataset.apply(
        lambda x: len(x['sequence_left_inspect']) + len(x['sequence_right_inspect']), axis=1)
    dataset_combined_length_binned = pd.cut(dataset_combined_length, [-1, 32, 64, 128, 256, 512, 50000],
                                            labels=['32', '64', '128', '256', '512', '50000'])
    print('Full sequence:')
    plt.hist(dataset_combined_length_binned)
    plt.show()

    dataset_combined_length = dataset.apply(
        lambda x: len(x['sequence_left_titleonly_inspect']) + len(x['sequence_right_titleonly_inspect']), axis=1)
    dataset_combined_length_binned = pd.cut(dataset_combined_length, [-1, 32, 64, 128, 256, 512, 50000],
                                            labels=['32', '64', '128', '256', '512', '50000'])
    print('Title only sequence:')
    plt.hist(dataset_combined_length_binned)
    plt.show()

    try:
        dataset_reduced = dataset[
            ['label', 'label_multi1', 'label_multi2', 'pair_id', 'sequence_left', 'sequence_right']]
        dataset_reduced_titleonly = dataset[
            ['label', 'label_multi1', 'label_multi2', 'pair_id', 'sequence_left_titleonly',
             'sequence_right_titleonly']].copy()
    except KeyError:
        dataset_reduced = dataset[['label', 'pair_id', 'sequence_left', 'sequence_right']]
        dataset_reduced_titleonly = dataset[
            ['label', 'pair_id', 'sequence_left_titleonly', 'sequence_right_titleonly']].copy()

    dataset_reduced_titleonly = dataset_reduced_titleonly.rename(columns={'sequence_left_titleonly': 'sequence_left',
                                                                          'sequence_right_titleonly': 'sequence_right'})

    dataset_inspect = dataset[
        ['sequence_left', 'sequence_left_inspect', 'sequence_left_titleonly', 'sequence_left_titleonly_inspect',
         'sequence_right', 'sequence_right_inspect', 'sequence_right_titleonly', 'sequence_right_titleonly_inspect',
         'pair_id']]

    return dataset_reduced, dataset_reduced_titleonly, dataset_inspect


In [ ]:
def _att_to_seq_amazongoogle(dataset):
    seq_left = dataset['manufacturer_left'] + ' ' + dataset['title_left'] + ' ' + ' ' + \
               dataset['price_left'].astype(str)
    seq_left_titleonly = dataset['manufacturer_left'] + ' ' + dataset['title_left']
    seq_right = dataset['manufacturer_right'] + ' ' + dataset['title_right'] + ' ' + \
                dataset['price_right'].astype(str)
    seq_right_titleonly = dataset['manufacturer_right'] + ' ' + dataset['title_right']
    return seq_left, seq_left_titleonly, seq_right, seq_right_titleonly